LCD域全称Leftover Capture Data，是USB协议中携带数据的主要部分  
可以近似类比为HTTP协议中的Body部分  
USB协议最大的特点在于，不同的设备有不同的交流协议，一般比赛时会提供网络允  
许你查询相关文档或者直接提供给你相关文档，考察你的学习能力  
Tshark使用过滤器导出数据 -Y “过滤表达式”  
```bash
 tshark -r usb1.pcap -T fields -e usb.capdata | sed '/^\s*$/d'> usbdata.txt
```

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

def main():
    global mousePositionX
    global mousePositionY
    # check argv
    if len(sys.argv) != 3:
        print("Usage : ")
        print("        python UsbMiceHacker.py data.pcap [LEFT|RIGHT|MOVE|ALL]")
        print("Tips : ")
        print("        To use this python script , you must install the numpy,matplotlib first.")
        print("        You can use `sudo pip install matplotlib numpy` to install it")
        print("Author : ")
        print("        WangYihang <wangyihanger@gmail.com>")
        print("        If you have any questions , please contact me by email.")
        print("        Thank you for using.")
        exit(1)

    # get argv
    pcapFilePath = sys.argv[1]
    action = sys.argv[2]

    # get data of pcap
    command = "tshark -r '%s' -T fields -e usb.capdata > '%s'" % (
        pcapFilePath, 'usbtmp')
    print(command)
    os.system(command)
    draw('usbtmp', action)


def draw(DataFileName, action):
    mousePositionX = 0
    mousePositionY = 0
    X = []
    Y = []
    data = []
    if action != "LEFT" and action != "ALL" and action != "RIGHT" and action != "MOVE":
        action = "LEFT"
    # read data
    with open(DataFileName, "r") as f:
        for line in f:
            data.append(line[0:-1])

    # handle move
    for i in data:
        try:
            Bytes = []
            for j in range(0, len(i), 2):
                Bytes.append(i[j:j + 2])
            if len(Bytes) == 8:
                horizontal = 2  # -
                vertical = 4  # |
            elif len(Bytes) == 4:
                horizontal = 1  # -
                vertical = 2  # |
            else:
                continue
            offsetX = int(Bytes[horizontal], 16)
            offsetY = int(Bytes[vertical], 16)
            if offsetX > 127:
                offsetX -= 256
            if offsetY > 127:
                offsetY -= 256
            mousePositionX += offsetX
            mousePositionY += offsetY
            if Bytes[0] == "01":
                print("<",end='')
                if action == "LEFT":
                    # draw point to the image panel
                    X.append(mousePositionX)
                    Y.append(-mousePositionY)
            elif Bytes[0] == "02":
                print(">",end='')
                if action == "RIGHT":
                    # draw point to the image panel
                    X.append(mousePositionX)
                    Y.append(-mousePositionY)
            elif Bytes[0] == "00":
                print("-",end='')
                if action == "MOVE":
                    # draw point to the image panel
                    X.append(mousePositionX)
                    Y.append(-mousePositionY)
            else:
                print("[-] Known operate.")
                pass
            if action == "ALL":
                # draw point to the image panel
                X.append(mousePositionX)
                Y.append(-mousePositionY)
        except:
            pass

    fig = plt.figure(figsize=(20,20))
    ax1 = fig.add_subplot(111)
    ax1.set_title('[%s]-[%s] Author : WangYihang' % ('Mouse:', action))
    ax1.scatter(X, Y, c='r', marker='.',s=1)
    plt.show()



draw('z:/ctf/a.txt', 'RIGHT')